# Analysis of CRDC referral and arrest data 

In [1]:
import re
import numpy as np
import pandas as pd


In [2]:
READ_ARGS = {"low_memory": False, "encoding": "latin"}

lower_cols = lambda col: col.lower()

enr = pd.read_csv("input/1718-crdc-sch-enrollment.csv", **READ_ARGS).rename(columns=lower_cols)
arr = pd.read_csv("input/1718-crdc-sch-referrals-arrests.csv", **READ_ARGS).rename(columns=lower_cols)
cha = pd.read_csv("input/1718-crdc-sch-characteristics.csv", **READ_ARGS).rename(columns=lower_cols)


## Overall rates

In [3]:
MAX_GRADE = 5
JOIN_ARGS = {"how": "left"}
INDEX_COLS = [
    "combokey",
    "sch_name",
    "lea_name",
    "lea_state_name",
    "jj",
]

sum_sch = lambda df, kwd, colname: (
    df.set_index(INDEX_COLS)[[c for c in df if "tot_" in c and kwd in c]]
    .sum(axis=1)
    .to_frame(colname)
    .query(f"{colname} > 0")
)


def get_max_grade(row):
    true_cols = [
        (k, v) for k, v in row.iteritems() if k.startswith("sch_grade_g") and v == "Yes"
    ]
    if len(true_cols) == 0:
        return np.NaN
    return int(re.search(r"(?<=sch_grade_g)\d{1,2}$", true_cols[-1][0]).group())


cha = cha.set_index(INDEX_COLS + ["sch_status_alt"]).assign(
    max_grade=lambda df: df.apply(get_max_grade, axis=1)
)[["max_grade"]]


MULT = 10_000
assign_rates = lambda df: (
    df.assign(arrest_rate=lambda df: df.arrests / df.enrollment * MULT).assign(
        referral_rate=lambda df: df.referrals / df.enrollment * MULT
    )
)


tot_df = (
    sum_sch(enr, "enr", "enrollment")
    .join(cha, **JOIN_ARGS)
    .join(sum_sch(arr, "arr", "arrests"), **JOIN_ARGS)
    .join(sum_sch(arr, "ref", "referrals"), **JOIN_ARGS)
    .query(
        "max_grade <= @MAX_GRADE & index.get_level_values('jj') == 'No' & index.get_level_values('sch_status_alt') == 'No'"
    )
    .pipe(assign_rates)
)

tot_df.head()


,,,,,,enrollment,max_grade,arrests,referrals,arrest_rate,referral_rate
combokey,sch_name,lea_name,lea_state_name,jj,sch_status_alt,,,,,,
010000500889,Albertville Elementary School,Albertville City,ALABAMA,No,No,1565,4.0,NaN,NaN,NaN,NaN
010000502150,Albertville Primary School,Albertville City,ALABAMA,No,No,1712,2.0,NaN,NaN,NaN,NaN
010000600876,Claysville School,Marshall County,ALABAMA,No,No,237,4.0,NaN,NaN,NaN,NaN
010000600877,Douglas Elementary School,Marshall County,ALABAMA,No,No,723,5.0,NaN,NaN,NaN,NaN
010000600880,Brindlee Mountain Elementary School,Marshall County,ALABAMA,No,No,257,5.0,NaN,NaN,NaN,NaN


### National

In [4]:
national_rates = (
    tot_df[["enrollment", "referrals", "arrests"]]
    .sum()
    .to_frame("United States")
    .transpose()
    .pipe(assign_rates)
    .iloc[0]
)

national_rates


enrollment       2.438163e+07
referrals        1.289200e+04
arrests          2.607000e+03
arrest_rate      1.069248e+00
referral_rate    5.287588e+00
Name: United States, dtype: float64

### By state

In [5]:
(
    tot_df.reset_index()
    .groupby("lea_state_name")[["enrollment", "arrests", "referrals"]]
    .sum()
    .pipe(assign_rates)
    .sort_values(["referral_rate", "arrest_rate"], ascending=False)
)


,enrollment,arrests,referrals,arrest_rate,referral_rate
lea_state_name,,,,,
VIRGINIA,692817,208.0,1840.0,3.002236,26.558240
PENNSYLVANIA,609573,44.0,972.0,0.721817,15.945588
NEW HAMPSHIRE,80051,25.0,126.0,3.123009,15.739966
ARIZONA,235820,44.0,328.0,1.865830,13.908914
MISSOURI,400470,225.0,557.0,5.618398,13.908657
WASHINGTON,571384,567.0,627.0,9.923274,10.973356
DELAWARE,77398,0.0,79.0,0.000000,10.206982
WISCONSIN,407558,33.0,377.0,0.809701,9.250217
FLORIDA,1634765,35.0,1306.0,0.214098,7.988916


### Individual schools

In [6]:
REFERRAL_RATE_THRESHOLD = national_rates.referral_rate * 10
ARREST_RATE_THRESHOLD = national_rates.arrest_rate * 2

problem_schools = (
    tot_df.pipe(assign_rates)
    .query(
        "enrollment >= 100 "
        "& referral_rate >= @REFERRAL_RATE_THRESHOLD"
        "& arrest_rate >= @ARREST_RATE_THRESHOLD"
    )
    .sort_values("referral_rate", ascending=False)
)

problem_schools.to_csv("output/schools.csv")

problem_schools


,,,,,,enrollment,max_grade,arrests,referrals,arrest_rate,referral_rate
combokey,sch_name,lea_name,lea_state_name,jj,sch_status_alt,,,,,,
530645002813,Ptarmigan Ridge Elementary School,Orting School District,WASHINGTON,No,No,965,5.0,484.0,483.0,5015.544041,5005.181347
290462000054,SCOTT ELEM.,BELTON 124,MISSOURI,No,No,293,4.0,60.0,61.0,2047.781570,2081.911263
260769004353,CLK Elementary School,Public Schools of Calumet Laurium & Keweenaw,MICHIGAN,No,No,541,5.0,67.0,67.0,1238.447320,1238.447320
530645000952,Orting Primary School,Orting School District,WASHINGTON,No,No,727,3.0,82.0,83.0,1127.922971,1141.678129
481662001427,SMITH EL,DEL VALLE ISD,TEXAS,No,No,1595,5.0,177.0,177.0,1109.717868,1109.717868
...,...,...,...,...,...,...,...,...,...,...,...
130183002314,Lamar Reese School of the Arts,Dougherty County,GEORGIA,No,No,537,5.0,2.0,3.0,37.243948,55.865922
510324000381,Barack Obama Elementary,RICHMOND CITY PBLC SCHS,VIRGINIA,No,No,359,5.0,2.0,2.0,55.710306,55.710306
550960002342,Zablocki Elementary,Milwaukee School District,WISCONSIN,No,No,719,5.0,2.0,4.0,27.816412,55.632823
